In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
files = []
y_list = []
classes = ['kiss', 'handshake', 'hug', 'highfive']

for class_n, c in enumerate(classes):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    filtered.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))
    
    #only use not-augmented videos
    filtered = filtered[:50]
    
    y_list.extend([class_n]*len(filtered))
    files.extend(filtered)

y = np.array(y_list)

x = [np.load(x)/255 for x in files]

In [ ]:
print(x[0].dtype)
assert len(x) == len(y)

In [ ]:
def split(X, y, test_size=0.1):
    shuffled_idx = np.random.permutation(len(X))
    
    split_at = int(len(shuffled_idx) * test_size)
    train_idx = shuffled_idx[split_at:]
    test_idx = shuffled_idx[:split_at]
    
    #TODO: add use of one-hot encoding
    onehot = np.zeros((y.size, 4))
    onehot[np.arange(y.size),y] = 1
    
    train_X = itemgetter(*train_idx)(X)
    train_y = itemgetter(*train_idx)(y)
    
    test_X = itemgetter(*test_idx)(X)
    test_y = itemgetter(*test_idx)(y)
    
    return (train_X, test_X, train_y, test_y)


train_X, test_X, train_y, test_y = split(x, y)
assert len(train_X) == len(train_y)

In [ ]:
epochs = 70
shuffled = list(zip(x, y))
shuffle(shuffled)

def batcherize(df):
    for _ in range(epochs):
        for features, labels in shuffled:
            _features = np.array([features])
            yield _features, np.atleast_1d(labels)
        
batcherize_train = batcherize(shuffled[:180])

def batcherize_t():
    for features, labels in shuffled[180:]:
        _features = np.array([features])
        yield _features, np.atleast_1d(labels)

In [ ]:
video = Input(shape=(None, #N_FRAMES
                     128, #WIDDTH
                     128, #HEIGHT
                     3), dtype='float')
cnn_base = VGG16(input_shape=(128,
                              128,
                              3),
                 weights="imagenet",
                 include_top=False)
cnn_out = GlobalAveragePooling2D()(cnn_base.output)

cnn = Model(cnn_base.input, cnn_out)
cnn.trainable = False

encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = LSTM(256)(encoded_frames)
hidden_layer = Dense(512, activation="relu")(encoded_sequence)
hidden_layer = Dense(256, activation="relu")(hidden_layer)
outputs = Dense(4, activation="softmax")(hidden_layer)
model = Model([video], outputs)

model.summary()


In [ ]:
optimizer =tf.keras.optimizers.Nadam(learning_rate=0.002,
                  beta_1=0.9,
                  beta_2=0.999,
                  epsilon=1e-08,
                  schedule_decay=0.004)

model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

test_df = tf.data.Dataset.from_generator(batcherize_t, output_types=(tf.float64, tf.int64))

h = model.fit(
    batcherize_train, 
    batch_size=1, 
    epochs=epochs, 
    steps_per_epoch=180, 
    validation_data=test_df, 
    validation_steps=20
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,4))

ax[0].plot(h.history['loss'], label='Training')
ax[0].plot(h.history['val_loss'], label='Test')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Categorical Crossentropy')
ax[0].set_title('Loss')
ax[0].legend()

ax[1].plot(h.history['accuracy'], label='Training')
ax[1].plot(h.history['val_accuracy'], label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')
ax[1].set_title('Accuracy')
ax[1].legend()

fig.savefig('./graphs/vgg/vgg_loss_accuracy.png', bbox_inches='tight', pad_inches=0.05)

In [ ]:
y_pred = model.predict(test_df, steps=20)

y_true = []

y_true = list(map(lambda x: int(x[1]), list(batcherize_t())))
y_true = np.array(y_true)

y_pred = np.argmax(y_pred, axis=1)

conf_m = sns.heatmap(tf.math.confusion_matrix(y_true, y_pred), annot=True, xticklabels=classes, yticklabels=classes)
conf_m.set(xlabel='Predicted labels', ylabel='True labels')

g.figure.savefig('./graphs/vgg/vgg_confusion_matrix.png', dpi=150, bbox_inches = "tight")